In [ ]:
!pip install seleniumbase 
!pip install selenium

In [1]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv

In [ ]:
class Leetcode:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.tag_list=[]
        self.companies_list=[]
        self.no = 0
        self.failed_list =[]

    def Login(self,id,pw):
        url = "https://leetcode.com/accounts/login/"
        self.driver.get(url)
        id_field = self.driver.find_element(by=By.ID, value="id_login")
        id_field.send_keys(id)
        time.sleep(1)
        pw_field = self.driver.find_element(by=By.ID, value="id_password")
        pw_field.send_keys(pw)
        #no need check ctp
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.ID, "signin_btn"))).click()
        time.sleep(5)
        #self.GetAllTagAndCompaniesList()
        self.GetTopHundredQuestion()

    def GetTopHundredQuestion(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Frequency')])[1]"))).click()
      
        table = self.driver.find_element(by = By.XPATH, value="(//div[@role='rowgroup'])")
        rows = table.find_elements(by = By.XPATH,value="(//div[@role='row'])")
        bool_skip_first = False
        for r in rows:
            if bool_skip_first == False:
                bool_skip_first = True
            else:
                col = r.find_elements(by = By.TAG_NAME, value="(//div[@role='cell'])")
                number = col[1].text
                acceptance = col[3].text
                difficulty = col[4].text
                print(number,acceptance,difficulty)
                #frequency = cols[6].get_attribute("value")
        

    def GetAllTagAndCompaniesList(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//button[@id='headlessui-popover-button-:r5:'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-blue dark:text-dark-blue m-1 flex cursor-pointer items-center px-1'][normalize-space()='Expand'])[1]"))).click()
        tag_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[1]")
        tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('tag.txt', 'w') as f:
            for e in tag_elements:
                self.tag_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-sm leading-5 font-normal text-label-2 dark:text-dark-label-2'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Expand')])[1]"))).click()
        companies_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[2]")
        companies_elements = companies_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('companies.txt', 'w') as f:
            for e in companies_elements:
                self.companies_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        time.sleep(2)
        self.GetCompaniesQuestion()

    def TextToWebText(self,text):
        if text == "Quip (Salesforce)":
            return "quip"

        if text == "Virtu Financial":
            return "virtu"
            
        text = text.lower()
        text = text.replace(' ', '-')
        text = text.replace('.', '')
        return text

    def GetCompaniesQuestion(self):
        dest_folder = 'Companies Leetcode'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        if self.no < len(self.companies_list):
            company_name = self.companies_list[self.no]
            file_name = company_name + ".csv"
            file_path = os.path.join(dest_folder, file_name)
            url = "https://leetcode.com/company/" + self.TextToWebText(company_name) + "/"
            self.driver.get(url)
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//input[@type='checkbox'])[1]"))).click()
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            table = self.driver.find_element(by = By.CSS_SELECTOR, value=".reactable-data")
            rows = table.find_elements(by = By.TAG_NAME, value="tr")
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Tags","Acceptance","Difficulty","Frequency"])

                error =False
                for r in rows:
                    cols = r.find_elements(by = By.TAG_NAME, value="td")
                    if len(cols) == 7:
                        no = cols[1].text
                        title = cols[2].text
                        tags_list = cols[3].find_elements(by = By.TAG_NAME, value="a")
                        tags = ""
                        for t in tags_list:
                            tags = tags + "[" + t.text + "]"
                        acceptance = cols[4].text
                        difficulty = cols[5].text
                        frequency = cols[6].get_attribute("value")
                        writer.writerow([no, title ,tags,acceptance,difficulty,frequency])
                    else:
                        error = True
                file.close()
            if error == True:
                self.failed_list.append(company_name)
                print("Data mining failed.",company_name)
            else:
                print("Done data mining",company_name)
                
            time.sleep(2)
            self.no += 1
            self.GetCompaniesQuestion()
        else:
            self.driver.quit()
                    


In [ ]:
id = 
pw =

In [ ]:
test = Leetcode()
test.Login(id,pw)


In [1]:
import csv
import os
import pandas as pd

In [6]:
class LeetcodeDataFrameConvert:
    def __init__(self):
        self.companies_list=[]
        self.all_tags_list=[]
        

    def InitTagsList(self):
        cf = open("tag.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.all_tags_list.append(c)
        cf.close

    
    def InitCompaniesList(self):
        cf = open("companies.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.companies_list.append(c)
        cf.close

    def ConvertToDataframe(self):
        dest_folder = 'Companies Leetcode DataFrame'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        for c in self.companies_list:
            read_filename = c
            read_filename +='.csv'
            read_file_path = os.path.join('Companies Leetcode', read_filename)
            df = pd.read_csv(read_file_path)
            #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
            #df = df.drop(columns=['Difficulty'])
            #df = pd.concat([df,difficulty], axis= 1)
            tags_header_list = []
            for index, row in df.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    if tag not in tags_header_list:
                        tags_header_list.append(tag)
            for tag in tags_header_list:
                df[tag] = 0
            df2 = df
            for index, row in df2.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    df.loc[index, tag] = 1
            df = df.drop(columns=['Tags'])
            file_name = c + ".csv"
            write_file_path = os.path.join(dest_folder, file_name)
            df.to_csv(write_file_path, encoding='utf-8', index=False)
            print(c,"file successfully converted to data frame format.")

    def ConvertToDataframeAllTag(self):
        self.InitTagsList()
        dest_folder = 'Companies Leetcode DataFrame All Tag'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        for c in self.companies_list:
            read_filename = c
            read_filename +='.csv'
            read_file_path = os.path.join('Companies Leetcode', read_filename)
            df = pd.read_csv(read_file_path)
            tags_header_list = self.all_tags_list
            for tag in tags_header_list:
                df[tag] = 0
            df2 = df
            for index, row in df2.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    df.loc[index, tag] = 1
            df = df.drop(columns=['Tags'])
            file_name = c + ".csv"
            write_file_path = os.path.join(dest_folder, file_name)
            df.to_csv(write_file_path, encoding='utf-8', index=False)
            print(c,"file successfully converted to data frame format.")

        
    

In [7]:
test_df = LeetcodeDataFrameConvert()
test_df.InitCompaniesList()
test_df.ConvertToDataframe()
#test_df.ConvertToDataframeAllTag()

Facebook file successfully converted to data frame format.
Google file successfully converted to data frame format.
Amazon file successfully converted to data frame format.
Microsoft file successfully converted to data frame format.
Uber file successfully converted to data frame format.
Apple file successfully converted to data frame format.
Oracle file successfully converted to data frame format.
TikTok file successfully converted to data frame format.
Goldman Sachs file successfully converted to data frame format.
Walmart Labs file successfully converted to data frame format.
Bloomberg file successfully converted to data frame format.
Adobe file successfully converted to data frame format.
DoorDash file successfully converted to data frame format.
Nvidia file successfully converted to data frame format.
LinkedIn file successfully converted to data frame format.
Citadel file successfully converted to data frame format.
Pinterest file successfully converted to data frame format.
Atlass